In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import os
import torch.optim as optim
from torchvision.utils import save_image
import os
from tqdm import tqdm

# -----------------------
# 1. Data Loading
# -----------------------
image_dir = "celebA/celeba/img_align_celeba"
os.makedirs("gan_outputs", exist_ok=True)

transform = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root=image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3, feature_g=64):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, feature_g * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(feature_g * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(feature_g * 8, feature_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(feature_g * 4, feature_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(feature_g * 2, feature_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g),
            nn.ReLU(True),
            nn.ConvTranspose2d(feature_g, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

class Critic(nn.Module):
    def __init__(self, img_channels=3, feature_d=64):
        super(Critic, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, feature_d, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(feature_d, feature_d * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(feature_d * 2, feature_d * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(feature_d * 4, feature_d * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_d * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(feature_d * 8, 1, 4, 1, 0, bias=False)
        )

    def forward(self, img):
        return self.net(img).view(-1)

# -----------------------
# 3. Training
# -----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

z_dim = 100
lr = 5e-5
n_epochs = 50
critic_iter = 5
weight_clip = 0.01

G = Generator(z_dim).to(device)
D = Critic().to(device)

optimizer_G = optim.RMSprop(G.parameters(), lr=lr)
optimizer_D = optim.RMSprop(D.parameters(), lr=lr)

fixed_noise = torch.randn(64, z_dim, 1, 1, device=device)

os.makedirs("wgan_outputs", exist_ok=True)

for epoch in range(n_epochs):
    g_loss_epoch = 0.0
    d_loss_epoch = 0.0
    num_batches = 0

    for i, (real_imgs, _) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{n_epochs}")):
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size(0)

        # Train Critic
        for _ in range(critic_iter):
            z = torch.randn(batch_size, z_dim, 1, 1, device=device)
            fake_imgs = G(z).detach()
            loss_D = -(D(real_imgs).mean() - D(fake_imgs).mean())

            optimizer_D.zero_grad()
            loss_D.backward()
            optimizer_D.step()

            for p in D.parameters():
                p.data.clamp_(-weight_clip, weight_clip)

        # Train Generator
        z = torch.randn(batch_size, z_dim, 1, 1, device=device)
        fake_imgs = G(z)
        loss_G = -D(fake_imgs).mean()

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        g_loss_epoch += loss_G.item()
        d_loss_epoch += loss_D.item()
        num_batches += 1

    # Save sample images
    with torch.no_grad():
        fake_samples = G(fixed_noise)
        save_image(fake_samples, f"wgan_outputs/epoch_{epoch+1:03d}.png", normalize=True)

    # Print average loss
    print(f"Epoch [{epoch+1}/{n_epochs}]  Loss_D: {d_loss_epoch/num_batches:.4f}  Loss_G: {g_loss_epoch/num_batches:.4f}")

# -----------------------
# 4. Generate 10,000 Images
# -----------------------
os.makedirs("wgan_outputs/generated", exist_ok=True)
G.eval()
with torch.no_grad():
    for i in tqdm(range(0, 10000, 64), desc="Generating final images"):
        z = torch.randn(64, z_dim, 1, 1, device=device)
        generated_imgs = G(z)
        for j in range(generated_imgs.size(0)):
            save_image(generated_imgs[j], f"wgan_outputs/generated/{i + j:05d}.png", normalize=True)

/sciclone/data10/iahewababarand/genVision-VAE/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


/sciclone/data10/iahewababarand/genVision-VAE/.venv/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/50:   0%|          | 0/3166 [00:00<?, ?it/s]

/sciclone/data10/iahewababarand/genVision-VAE/.venv/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/50:   0%|          | 1/3166 [00:07<6:52:16,  7.82s/it]

...

Epoch 2/50:  96%|█████████▋| 3049/3166 [5:12:30<12:01,  6.17s/it]

Epoch 2/50:  96%|█████████▋| 3050/3166 [5:12:36<11:52,  6.14s/it]

Epoch 2/50:  96%|█████████▋| 3051/3166 [5:12:42<11:43,  6.12s/it]

Epoch 2/50:  96%|█████████▋| 3052/3166 [5:12:48<11:35,  6.10s/it]

Epoch 2/50:  96%|█████████▋| 3053/3166 [5:12:54<11:26,  6.07s/it]